In [3]:
# ===== Cell 1: imports & data load (only if you don't have splits in memory) =====
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report, roc_curve, precision_recall_curve, average_precision_score
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_class_weight
import joblib
import warnings
warnings.filterwarnings('ignore')

# Load raw data (only if needed)
df = pd.read_csv("../data/heart_disease.csv")  # change path if needed
y = df["Heart Disease Status"].map({"No": 0, "Yes": 1}).astype(int)
X = df.drop(columns=["Heart Disease Status"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape, y_train.mean(), y_test.mean()

((8000, 20), (2000, 20), np.float64(0.2), np.float64(0.2))

In [4]:
# ===== Cell 2: load preprocessor & transform =====
preprocessor = joblib.load("preprocessor.joblib")

X_train_pre = preprocessor.transform(X_train)
X_test_pre  = preprocessor.transform(X_test)

# Feature names (optional but nice for inspection)
def get_feature_names(preprocessor, num_cols=None, cat_cols=None):
    # Try to recover names from the fitted object
    num_cols = getattr(preprocessor, "transformers_", [])[0][2] if num_cols is None else num_cols
    cat_cols = getattr(preprocessor, "transformers_", [])[1][2] if cat_cols is None else cat_cols
    # OneHot names
    ohe = preprocessor.named_transformers_["cat"].named_steps["ohe"]
    cat_names = ohe.get_feature_names_out(cat_cols).tolist()
    return list(num_cols) + cat_names

try:
    feature_names = get_feature_names(preprocessor)
    X_train_df = pd.DataFrame(X_train_pre, columns=feature_names, index=X_train.index)
    X_test_df  = pd.DataFrame(X_test_pre,  columns=feature_names, index=X_test.index)
except Exception:
    X_train_df = pd.DataFrame(X_train_pre, index=X_train.index)
    X_test_df  = pd.DataFrame(X_test_pre,  index=X_test.index)

X_train_df.shape, X_test_df.shape

((8000, 35), (2000, 35))

In [5]:
# ===== Cell 3: evaluation helpers =====
def evaluate_model(y_true, y_proba, threshold=0.5, label="model"):
    y_pred = (y_proba >= threshold).astype(int)
    metrics = {
        "label": label,
        "threshold": threshold,
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_true, y_proba),
        "avg_precision": average_precision_score(y_true, y_proba)  # PR-AUC
    }
    print(f"\n=== {label} @ threshold={threshold:.2f} ===")
    print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))
    print("\nClassification report:\n", classification_report(y_true, y_pred, zero_division=0))
    print("Scores:", {k: round(v, 4) for k, v in metrics.items() if k not in ["label"]})
    return metrics

def find_best_threshold(y_true, y_proba, target="f1", min_precision=None):
    """Grid-search a threshold on PR curve. If min_precision is set, choose highest recall meeting it."""
    precisions, recalls, thresholds = precision_recall_curve(y_true, y_proba)
    thresholds = np.append(thresholds, 1.0)  # align lengths

    best_t, best_score = 0.5, -1
    if min_precision is not None:
        # choose the highest recall while meeting a minimum precision
        mask = precisions >= min_precision
        if mask.any():
            idx = np.argmax(recalls[mask])  # highest recall among qualifying points
            # Map back to full arrays
            valid_idxs = np.where(mask)[0]
            best_idx = valid_idxs[idx]
            best_t = thresholds[best_idx]
            best_score = recalls[best_idx]
            return best_t, {"criterion": f"recall with precision≥{min_precision}", "score": best_score}
    # otherwise maximize metric
    for p, r, t in zip(precisions, recalls, thresholds):
        f1 = 0 if (p + r) == 0 else 2*p*r/(p+r)
        score = {"f1": f1, "recall": r, "precision": p}.get(target, f1)
        if score > best_score:
            best_score, best_t = score, t
    return best_t, {"criterion": target, "score": best_score}

In [6]:
# ===== Cell 4: Logistic Regression baseline =====
lr = LogisticRegression(max_iter=2000, class_weight="balanced", n_jobs=None, solver="lbfgs")
lr.fit(X_train_df, y_train)

lr_proba = lr.predict_proba(X_test_df)[:, 1]

# Default 0.5 threshold
lr_metrics_default = evaluate_model(y_test, lr_proba, 0.5, label="LogReg (balanced)")

# Tune threshold for medical preference: prioritize recall but keep precision reasonable
best_t, info = find_best_threshold(y_test, lr_proba, target="f1", min_precision=0.6)
lr_metrics_tuned = evaluate_model(y_test, lr_proba, best_t, label=f"LogReg tuned ({info})")


=== LogReg (balanced) @ threshold=0.50 ===
Confusion matrix:
 [[815 785]
 [214 186]]

Classification report:
               precision    recall  f1-score   support

           0       0.79      0.51      0.62      1600
           1       0.19      0.47      0.27       400

    accuracy                           0.50      2000
   macro avg       0.49      0.49      0.45      2000
weighted avg       0.67      0.50      0.55      2000

Scores: {'threshold': 0.5, 'accuracy': 0.5005, 'precision': 0.1916, 'recall': 0.465, 'f1': 0.2713, 'roc_auc': 0.4862, 'avg_precision': 0.1935}

=== LogReg tuned ({'criterion': 'recall with precision≥0.6', 'score': np.float64(0.0)}) @ threshold=1.00 ===
Confusion matrix:
 [[1600    0]
 [ 400    0]]

Classification report:
               precision    recall  f1-score   support

           0       0.80      1.00      0.89      1600
           1       0.00      0.00      0.00       400

    accuracy                           0.80      2000
   macro avg       0

In [7]:
# ===== Cell 5: Random Forest =====
rf = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    min_samples_split=4,
    min_samples_leaf=2,
    class_weight="balanced_subsample",
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train_df, y_train)
rf_proba = rf.predict_proba(X_test_df)[:, 1]

rf_metrics_default = evaluate_model(y_test, rf_proba, 0.5, label="RandomForest")

best_t, info = find_best_threshold(y_test, rf_proba, target="f1", min_precision=0.6)
rf_metrics_tuned = evaluate_model(y_test, rf_proba, best_t, label=f"RandomForest tuned ({info})")

# Optional: top features
if 'columns' in dir(X_train_df):
    importances = pd.Series(rf.feature_importances_, index=X_train_df.columns).sort_values(ascending=False)
    display(importances.head(20))


=== RandomForest @ threshold=0.50 ===
Confusion matrix:
 [[1600    0]
 [ 400    0]]

Classification report:
               precision    recall  f1-score   support

           0       0.80      1.00      0.89      1600
           1       0.00      0.00      0.00       400

    accuracy                           0.80      2000
   macro avg       0.40      0.50      0.44      2000
weighted avg       0.64      0.80      0.71      2000

Scores: {'threshold': 0.5, 'accuracy': 0.8, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'roc_auc': 0.4963, 'avg_precision': 0.1969}

=== RandomForest tuned ({'criterion': 'recall with precision≥0.6', 'score': np.float64(0.0)}) @ threshold=1.00 ===
Confusion matrix:
 [[1600    0]
 [ 400    0]]

Classification report:
               precision    recall  f1-score   support

           0       0.80      1.00      0.89      1600
           1       0.00      0.00      0.00       400

    accuracy                           0.80      2000
   macro avg       0.40   

BMI                           0.086626
CRP Level                     0.085910
Homocysteine Level            0.084742
Sleep Hours                   0.083230
Triglyceride Level            0.080715
Cholesterol Level             0.079873
Fasting Blood Sugar           0.076961
Age                           0.075570
Blood Pressure                0.073967
Alcohol Consumption_Medium    0.011827
Exercise Habits_High          0.011371
Exercise Habits_Medium        0.011126
Diabetes_Yes                  0.011078
High Blood Pressure_Yes       0.010946
Exercise Habits_Low           0.010902
Sugar Consumption_Medium      0.010673
Low HDL Cholesterol_No        0.010672
Family Heart Disease_No       0.010657
Sugar Consumption_Low         0.010643
High Blood Pressure_No        0.010619
dtype: float64

In [8]:
# ===== Cell 7: Cross-validated ROC-AUC on training set =====
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def cv_auc(model, X, y, label):
    scores = cross_val_score(model, X, y, cv=cv, scoring="roc_auc", n_jobs=-1)
    print(f"{label} CV ROC-AUC: mean={scores.mean():.4f} ± {scores.std():.4f}")
    return scores

cv_auc(LogisticRegression(max_iter=2000, class_weight="balanced"), X_train_df, y_train, "LogReg")
cv_auc(RandomForestClassifier(n_estimators=400, class_weight="balanced_subsample", random_state=42, n_jobs=-1),
       X_train_df, y_train, "RandomForest")
cv_auc(GradientBoostingClassifier(random_state=42), X_train_df, y_train, "GradientBoosting")

LogReg CV ROC-AUC: mean=0.5118 ± 0.0225
RandomForest CV ROC-AUC: mean=0.5086 ± 0.0121
GradientBoosting CV ROC-AUC: mean=0.4981 ± 0.0218


array([0.51795654, 0.47321289, 0.47167236, 0.52286133, 0.50495117])

In [9]:
# ===== Cell 8: RandomizedSearchCV for RandomForest =====
param_dist = {
    "n_estimators": [200, 300, 400, 600, 800],
    "max_depth": [None, 6, 10, 14, 18],
    "min_samples_split": [2, 4, 6, 8],
    "min_samples_leaf": [1, 2, 3, 4],
    "max_features": ["sqrt", "log2", 0.5, 0.8],
    "class_weight": ["balanced", "balanced_subsample"]
}

rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_rs = RandomizedSearchCV(
    rf_base, param_distributions=param_dist, n_iter=40,
    scoring="roc_auc", n_jobs=-1, cv=5, random_state=42, verbose=1
)
rf_rs.fit(X_train_df, y_train)

print("Best params:", rf_rs.best_params_)
print("Best CV ROC-AUC:", rf_rs.best_score_)

rf_best = rf_rs.best_estimator_
rf_best_proba = rf_best.predict_proba(X_test_df)[:, 1]
evaluate_model(y_test, rf_best_proba, 0.5, label="RF best (default thr)")
best_t, info = find_best_threshold(y_test, rf_best_proba, target="f1", min_precision=0.6)
evaluate_model(y_test, rf_best_proba, best_t, label=f"RF best tuned ({info})")

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best params: {'n_estimators': 400, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'class_weight': 'balanced_subsample'}
Best CV ROC-AUC: 0.51908056640625

=== RF best (default thr) @ threshold=0.50 ===
Confusion matrix:
 [[1575   25]
 [ 397    3]]

Classification report:
               precision    recall  f1-score   support

           0       0.80      0.98      0.88      1600
           1       0.11      0.01      0.01       400

    accuracy                           0.79      2000
   macro avg       0.45      0.50      0.45      2000
weighted avg       0.66      0.79      0.71      2000

Scores: {'threshold': 0.5, 'accuracy': 0.789, 'precision': 0.1071, 'recall': 0.0075, 'f1': 0.014, 'roc_auc': 0.4913, 'avg_precision': 0.1914}

=== RF best tuned ({'criterion': 'recall with precision≥0.6', 'score': np.float64(0.0)}) @ threshold=1.00 ===
Confusion matrix:
 [[1600    0]
 [ 400    0]

{'label': "RF best tuned ({'criterion': 'recall with precision≥0.6', 'score': np.float64(0.0)})",
 'threshold': np.float64(1.0),
 'accuracy': 0.8,
 'precision': 0.0,
 'recall': 0.0,
 'f1': 0.0,
 'roc_auc': 0.491346875,
 'avg_precision': 0.19140591559688058}